In [2]:
import pandas as pd


liste_dico = [{"comp_id" : 549,
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"comp_id" : 393,
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"comp_id" : 243,
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]
dico = liste_dico[0]

In [4]:
data_import = pd.read_excel(f"../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_pressure.xlsx", index_col = 0)

In [10]:
data = data_import.set_index("team_name")
data = data[data.quality_check == True]
data.fillna(0, inplace = True)

nb_matchs = pd.Series(index = data.index.unique())
for team in nb_matchs.index :
     nb_matchs[team] = len(data.loc[team].match_id.unique())
nb_matchs = nb_matchs.reindex(dico["ranking"])

drop = ["quality_check", "player_id", "player_name", "short_name", "player_birthdate", "match_name", "match_date", "team_id",
          "competition_id", "competition_name", "season_id", "season_name", "competition_edition_id", "position", "group", "result", "venue",
          "third", "channel", "adjusted_min_tip_per_match"]
data.drop(drop, inplace = True, axis = 1)
sample = data.columns[["sample" in i for i in data.columns]]
data.drop(sample, inplace = True, axis = 1)

col_ratio_rep = data.columns[[("ratio" in i) and ("per_100_pressures" in i or "per_match" in i) for i in data.columns]]
data.drop(col_ratio_rep, inplace = True, axis = 1)


In [12]:
data.columns.tolist()

['match_id',
 'minutes_played_per_match',
 'count_low_pressures_received_per_match',
 'count_forced_losses_under_low_pressure_per_match',
 'count_ball_retentions_under_low_pressure_per_match',
 'count_pass_attempts_under_low_pressure_per_match',
 'count_completed_passes_under_low_pressure_per_match',
 'count_dangerous_pass_attempts_under_low_pressure_per_match',
 'count_completed_dangerous_passes_under_low_pressure_per_match',
 'count_difficult_pass_attempts_under_low_pressure_per_match',
 'count_completed_difficult_passes_under_low_pressure_per_match',
 'count_medium_pressures_received_per_match',
 'count_forced_losses_under_medium_pressure_per_match',
 'count_ball_retentions_under_medium_pressure_per_match',
 'count_pass_attempts_under_medium_pressure_per_match',
 'count_completed_passes_under_medium_pressure_per_match',
 'count_dangerous_pass_attempts_under_medium_pressure_per_match',
 'count_completed_dangerous_passes_under_medium_pressure_per_match',
 'count_difficult_pass_attempt

In [20]:
data_col = data.columns[["full_otip_" in i for i in data.columns]]

In [29]:
def rename_col(col) :
    if "full_all" in col :
        return col.replace("full_all", "per_Match")
    elif "full_otip_p30otip" in col :
        return col.replace("full_otip_p30otip", "per30otip")
    elif "full_otip_p30tip" in col :
        return col.replace("full_otip_p30tip", "per30tip")
    else :
        return col

In [30]:
data.rename(columns = rename_col)

,match_id,minutes_per_Match,total_distance_per_Match,total_metersperminute_per_Match,running_distance_per_Match,hsr_distance_per_Match,hsr_count_per_Match,sprint_distance_per_Match,sprint_count_per_Match,hi_distance_per_Match,...,hsr_distance_per30otip,hsr_count_per30otip,sprint_distance_per30otip,sprint_count_per30otip,hi_distance_per30otip,hi_count_per30otip,medaccel_count_per30otip,highaccel_count_per30otip,meddecel_count_per30otip,highdecel_count_per30otip
team_name,,,,,,,,,,,,,,,,,,,,,
Angers SCO,1007272,97.37,9583.0,98.42,1230.0,355.0,39.0,145.0,8.0,500.0,...,378.03,34.89,79.97,4.36,458.00,39.26,65.43,4.36,40.71,7.27
Angers SCO,1021419,98.10,10355.0,105.56,1074.0,241.0,23.0,90.0,5.0,331.0,...,142.41,14.90,77.83,4.97,220.24,19.87,51.33,3.31,33.12,3.31
Angers SCO,1035154,102.35,10006.0,97.76,901.0,289.0,33.0,83.0,7.0,372.0,...,198.38,19.05,32.50,3.36,230.88,22.42,49.32,1.12,32.50,2.24
Angers SCO,1050074,99.27,10528.0,106.06,1054.0,273.0,27.0,60.0,4.0,333.0,...,231.11,20.00,36.67,1.11,267.78,21.11,54.44,2.22,35.56,4.44
Angers SCO,1064337,99.68,9441.0,94.71,876.0,186.0,22.0,34.0,3.0,220.0,...,149.13,14.60,16.69,2.09,165.82,16.69,51.10,2.09,33.37,6.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Girondins de Bordeaux,1543780,26.77,3076.0,114.92,436.0,139.0,19.0,57.0,3.0,196.0,...,364.93,54.25,152.88,4.93,517.81,59.18,49.32,0.00,39.45,9.86
Girondins de Bordeaux,1527200,10.67,1261.0,118.22,187.0,123.0,6.0,19.0,1.0,142.0,...,659.73,24.43,65.16,0.00,724.89,24.43,40.72,0.00,24.43,0.00
ES Troyes AC,1539379,12.60,1556.0,123.49,204.0,142.0,12.0,15.0,2.0,157.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [21]:
[j.replace("full_otip_", "") for j in data_col]

['total_distance_p30otip',
 'total_metersperminute_p30otip',
 'running_distance_p30otip',
 'hsr_distance_p30otip',
 'hsr_count_p30otip',
 'sprint_distance_p30otip',
 'sprint_count_p30otip',
 'hi_distance_p30otip',
 'hi_count_p30otip',
 'medaccel_count_p30otip',
 'highaccel_count_p30otip',
 'meddecel_count_p30otip',
 'highdecel_count_p30otip']

In [22]:
data.columns[["full_otip_" in i for i in data.columns]] = [j.replace("full_otip_", "") for j in data_col]

TypeError: Index does not support mutable operations